# Modelos e Estratégias de Trading
## Ajustando um modelo VAR à PETR4

Uirá Caiado. 05 de Setembro, 2016

**Resumo**

*Uma das principais necessidades quando falamos trading é realizar algum tipo de previsão sobre o estado futuro do instrumento operado para que possamos posicionar nossas ofertas. Como um dos objetivos da análise de séries temporais multivariadas é a realização de forecasts, nesta atividade vou implementar um modelo desta classe chamado Vetor Auto Regressivo (VAR). Vou ajustar o modelo aos retornos e informações de book da PETR4 e analisar a performance realizando um back-test em uma parte dos dados coletados que ainda não tenha sido utilizada.*

## 1. Introdução

Nesta sessão vou ...


### 1.1. Vetor Auto Regressivo (VAR)

bla bla

Pressupostos



### 1.2. O Problema
bla bla

## 2. Implementando o Modelo

Nesta sessão detalharemos e implementaremos ...

### 2.1. Estimando os Parâmetros

bla bla

In [2]:
import pandas as pd
df = pd.read_excel('data/VAR3_inputs_e_outputs.xlsx', header=None)
l_index = pd.date_range(start= '01/01/2014', end='01/12/2016')
df.index = l_index[:df.shape[0]]

In [3]:
from statsmodels.tsa.api import VAR
import statsmodels.tsa.vector_ar.util as util
model = VAR(df)
results = model.fit(3)
aux = util.get_var_endog(df.values, 3)

In [4]:
import var_model.vector_autoregression as var
reload(var)
self = var.VectorAutoregression(df)
self.fit(3)

In [67]:
# calcula erros para implementacao do statsmodel
print "error in Z: {:0.8f}".format(sum(sum(abs(self.na_Z.T - aux))))
print "error in A: {:0.8f}".format(sum(sum(sum(abs(self.na_A - results.coefs)))))
print "error in B: {:0.8f}".format(sum(sum(abs(self.na_betahat - results.params.values))))
print "error in Sigma U: {:0.8f}".format(sum(sum(abs(self.na_Sigma - results.sigma_u.values))))

error in Z: 0.00000000
error in A: 0.00000000
error in B: 0.00000000
error in Sigma U: 0.00000000


### 2.2. Forecast

Segundo notas de aula, para se afirmar o que ocorrerá no futuro $\left (  y_1, ..., y_k \right )$, tendo um processo $VAR(p)$ ajustado a um conjunto de dados $\Omega_t = \{ y_s \mid s \leq t \}$ para um horizonte de tempo $h$, precisamos determinar qual o *forecast* ótimo determinando aquele que minimiza uma função custo associada à seus erros (quadráticos médios - MSE na sigla em inglês). O preditor que minimiza estes erros é a esperança condicional (Lutkepohl, p. 33)

$$\mathbf{E}\left [ y_{t+h} \right ] := \mathbf{E}\left [ y_{t+h} \mid  \Omega_t \right ] = \mathbf{E}\left [ y_{t+h} \mid  \{ y_s \mid s \leq t \} \right ]$$

Lutkepohl ainda demonstra que a otimização da esperança condicional impica que:

$$
\mathbf{E}\left [ y_{t+1} \right ] = \upsilon + A_1 y_t + ... + A_p y_{t-p + 1} \\
\mathbf{E}\left [ y_{t+2} \right ] = \upsilon + A_1 \mathbf{E}\left [ y_{t+1} \right ] + A_2 y_t + ... + A_p y_{t-p + 2} \\
\vdots
$$


### 2.3. Intervalo de Confiança

Segundo Lutkepohl (p. 38), o erro de previsão (e consequentemente, o intervalo de confiança) pode ser obtido através da matriz de covariância dos erros (ou, da sigla em inglês, MSE matrix). Porém, quando se deseja realizar *forecasts* para mais de um período, é necessário definir também a matriz de coeficientes de Média Móvel (MA), que para um VAR(2) fica:

$$
\phi_1 = A_1 \\
\phi_2 = \phi_1 A_1 + A2 \\
\phi_3 = \phi_2 A_1 + \phi_1 A2 \\
\vdots \\
\phi_i = \phi_{i-1} A_1 + \phi_{i-2} A_2
$$

Assim, a matriz MSE de *forecast* são obtidas recursivamente aplicando

$$
\Gamma_y(0) = \Sigma_y = \sum_{i=0}^{\infty} \phi_i \Sigma_u \phi_i^{'}
$$

Sendo que:
$$
\Sigma_y (1) = \Sigma_u \\
\Sigma_y (2) = \Sigma_u + \phi_{1} \Sigma_u \phi_{1}' \\
\Sigma_y (3) = \Sigma_y (2) + \phi_{2} \Sigma_u \phi_{2}' \\
\vdots
$$

O $h$ em $\Sigma_y (h)$ se refere a quantos passos para frente se aplica a função de forecast. Como o modelo de VAR assume que os erros $u_t \sim N \left( 0, \, \Sigma_y(h) \right)$, podemos assumir que o erro de *forecast* também é normalmente distribuído. Com este pressuposto, podemos definir um [intervalo de confiança](https://en.wikipedia.org/wiki/Confidence_interval) na forma: 
$$\left[ y_{k, \, t} (h) - z_{(\alpha / 2)} \sigma_k (h), \, \, \, \, \, y_{k, \, t} (h) + z_{(\alpha / 2)} \sigma_k (h) \right]$$

Onde $\sigma(h)$ é a raiz quadrada do ** *k-ésimo* elemento da diagonal** $\Sigma_y(h)$ de maneira que

TODO: escrever exemplo da página 38 para não me perder depois

In [26]:
import var_model.vector_autoregression as var
import numpy as np
import pandas as pd
reload(var)
self = var.VectorAutoregression(df)
self.fit(3)

In [52]:
my_forecast, my_max, my_min = self.forecast(df[-5:].values, 3)
na_forecast, na_min, na_max = results.forecast_interval(df[-3:].values, 3)

In [69]:
print "Erro no limite inferior: {:0.7f}".format(sum(abs(na_min[-1] - my_min)))
print "Erro no Forecast: {:0.7f}".format(sum(abs(na_forecast[-1] - my_forecast)))
print "Erro no limite superior: {:0.7f}".format(sum(abs(na_max[-1] - my_max)))
print '\n\n'
df_plot = pd.DataFrame([my_min, na_min[-1]], index=['meu modelo', 'statsmodel'])
df_plot.index.name = 'Minimo'
print str(df_plot)
print ''

df_plot = pd.DataFrame([my_max, na_max[-1]], index=['meu modelo', 'statsmodel'])
df_plot.index.name = 'Maximo'
print df_plot

Erro no limite inferior: 0.0030550
Erro no Forecast: 0.0000000
Erro no limite superior: 0.0030550



                   0         1         2         3         4
Minimo                                                      
meu modelo -0.030019 -0.035569 -0.037285 -0.034497 -0.046392
statsmodel -0.030496 -0.036200 -0.037916 -0.035087 -0.047116

                   0         1         2         3         4
Maximo                                                      
meu modelo  0.026364  0.033363  0.034616  0.032315  0.040077
statsmodel  0.026841  0.033994  0.035248  0.032906  0.040801


### 2.4. Selecionando a Ordem do VAR

bla bla

## 3. Aplicando o modelo VAR

Nesta seção ...


## 4. Conclusão

bla bla




## 5. Últimas Considerações

bla bla


*Style notebook and change matplotlib defaults*

In [9]:
#loading style sheet
from IPython.core.display import HTML
HTML(open('ipython_style.css').read())

In [6]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set2", 10))